In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import folium
from folium import Popup

In [2]:
sewage = pd.read_csv('SewageLeaksRefined.csv')
# Convert Date Columns to datetime
sewage["Noted"] = pd.to_datetime(sewage["Noted"], format = "%d/%m/%Y")
sewage["Resolved"] = pd.to_datetime(sewage["Resolved"], format = "%d/%m/%Y")

colours = ['gray','blue', 'green', 'orange', 'red', 'darkred']

In [3]:
# Real world latitudes and longitudes that cover Lobitos
lobitos_max_lat = -4.449793440238931
lobitos_min_lat = -4.460475724172084
lobitos_min_lon = -81.28962377680455
lobitos_max_lon = -81.2738937464837

# Set size of grid width and length: E.g 10x10 square, 100 sectors
grid_size = 10

# Generate center points for each Lobitos sector
lobitos_lat_center = np.linspace(lobitos_min_lat, lobitos_max_lat, grid_size)
lobitos_lon_center = np.linspace(lobitos_min_lon, lobitos_max_lon, grid_size)

# Make even lat-lon squares around each Lobitos sector center point 
lat_diffs = (lobitos_lat_center[1] - lobitos_lat_center[0])/2 # Lat difference between two center points
lobitos_lat_min = lobitos_lat_center - lat_diffs # Find Lat min by taking the Lat diff from the center point
lobitos_lat_max = lobitos_lat_center + lat_diffs # Find Lat max by adding the Lat diff to the center point

lon_diffs = (lobitos_lon_center[1] - lobitos_lon_center[0])/2
lobitos_lon_min = lobitos_lon_center - lon_diffs
lobitos_lon_max = lobitos_lon_center + lon_diffs


# Make an empty dataframe to contain all the lat/lon data required for each sector
lobitos_sectors = pd.DataFrame(columns=['sector','center_lat','center_lon','lat_min','lat_max','lon_min','lon_max'])

# Loop through grid to assign lat/lon data for each sector
n = 0
for i in range(grid_size):
    for j in range(grid_size):
        lobitos_sectors.loc[n] = [f"Sector {n}"] + [lobitos_lat_center[i]] + [lobitos_lon_center[j]] + [lobitos_lat_min[i]] + [lobitos_lat_max[i]] + [lobitos_lon_min[j]] + [lobitos_lon_max[j]]
        n += 1

In [4]:
# Assign a sewage leak to the sector its lat/lon falls within
def assign_sector(row):
    mask = (
        (row['Lat'] >= lobitos_sectors['lat_min']) &
        (row['Lat'] <= lobitos_sectors['lat_max']) &
        (row['Lon'] >= lobitos_sectors['lon_min']) &
        (row['Lon'] <= lobitos_sectors['lon_max'])
    )

    matching_sectors = lobitos_sectors[mask]
    if len(matching_sectors) > 0:
        return matching_sectors.iloc[0]['sector']
    return 'Other'

# Example of allowing for further filtering of selection e.g. by date range e.g by admin selection in web app
# startDateFilter = pd.to_datetime('01/01/2024', format ="%d/%m/%Y")
# endDateFilter = pd.to_datetime('01/05/2024', format ="%d/%m/%Y")
# sewage = sewage[
#     (sewage['Noted'] >= startDateFilter) &
#     (sewage['Noted'] <= endDateFilter)
# ]

# Example of allowing for further filtering of selection e.g. by issue type e.g by admin selection in web app
# issueTypeFilter = "Leakage"
# sewage = sewage[(sewage['IssueType'] == issueTypeFilter)]


sewage.head(50)
# Loop through sewage DF assigning a sector for each incident
sewage['coord_sector'] = sewage.apply(assign_sector, axis=1)

# Group by assigned sector and calculate metrics
sector_counts = sewage.groupby('coord_sector').agg({
    'Name': 'count',
    'Diameter': ['sum', 'mean'],  # Adding sum and mean of diameter,
    'Severity': ['sum', 'mean']   # Adding sum and mean of severity,
}).reset_index()

# Flatten column names
sector_counts.columns = ['sector', 'count', 'total diameter', 'average diameter', 'total severity', 'average severity']

# Merge with sector coordinates
lobitos_data = sector_counts.merge(
    lobitos_sectors[['sector', 'center_lat', 'center_lon']],
    how='left',
    on='sector'
)

# Filter data which falls outside the sectors
lobitos_data = lobitos_data[(lobitos_data['sector'] != 'Other')]

sewage.head(50)

,Name,Lat,Lon,Noted,Resolved,Duration,Diameter,Severity,IssueType,coord_sector
0,Sewage Pool 1,-4.454175,-81.286566,2024-01-01,2024-04-01,91,5.786188,2,Leakage,Sector 52
1,Sewage Pool 2,-4.460881,-81.276439,2024-02-27,2024-10-17,233,1.536297,3,Leakage,Sector 8
2,Sewage Pool 3,-4.457939,-81.280830,2024-05-26,2024-08-02,68,6.023900,5,Damaged Pipe,Sector 25
3,Sewage Pool 4,-4.460197,-81.286455,2024-05-24,2024-11-12,172,9.876951,4,Blockage,Sector 2
4,Sewage Pool 5,-4.454613,-81.286012,2024-03-05,2024-10-18,227,2.188169,2,Blockage,Sector 52
5,Sewage Pool 6,-4.460457,-81.285773,2024-03-17,2024-10-18,215,7.047740,5,Leakage,Sector 2
6,Sewage Pool 7,-4.455902,-81.276937,2024-05-03,2024-12-16,227,1.387682,5,Damaged Pipe,Sector 47
7,Sewage Pool 8,-4.459192,-81.279181,2024-04-02,2024-08-25,145,2.177705,5,Overflow,Sector 16
8,Sewage Pool 9,-4.456092,-81.286291,2024-05-02,2024-10-10,161,3.130829,5,Damaged Pipe,Sector 42
9,Sewage Pool 10,-4.456277,-81.285714,2024-03-21,2024-11-04,228,6.466059,5,Leakage,Sector 42


In [5]:
def assign_colour_on_leaks(leaks):
    if leaks >= 5:
        return colours[5]
    elif leaks >= 4:
        return colours[4]
    elif leaks >= 3:
        return colours[3]
    elif leaks >= 2:
        return colours[2]
    elif leaks >= 1:
        return colours[1]
    else:
        return colours[0]

def assign_colour_on_severity(severity):
    if severity >= 5:
        return colours[5]
    elif severity >= 4:
        return colours[4]
    elif severity >= 3:
        return colours[3]
    elif severity >= 2:
        return colours[2]
    elif severity >= 1:
        return colours[1]
    else:
        return colours[0]
        
# Set Folium start point at Lobitos location
lobitosGrid = folium.Map(location=[-4.457310481797269, -81.2811891931266], zoom_start=16)

# Add the sectors as rectangles on the Folium map
for _, row in lobitos_sectors.iterrows():
    kw = {
        "color": "blue",
        "line_cap": "round",
        "fill": True,
        "fill_color": "white",
        "weight": 1,
        "tooltip": row['sector']
    }
    folium.Rectangle(
        bounds=[[row['lat_min'], row['lon_min']], [row['lat_max'], row['lon_max']]],
        line_join="round",
        dash_array="5, 5",
        **kw,
    ).add_to(lobitosGrid)


for _, row in lobitos_data.iterrows():
    # Scale the radius based on leak count (adjust multiplier as needed to give appropriate bubble sizes)
    radius = np.sqrt(row['count']) * 2

    tooltip_text = folium.Tooltip(f"""
    <b>{row['sector']}</b><br>
    Number of Leaks: {row['count']}<br>
    Total Diameter: {row['total diameter']:,.2f}m<br>
    Average Diameter: {row['average diameter']:,.2f}m<br>
    Total Severity: {row['total severity']:,.2f}<br>
    Average Severity: {row['average severity']:,.2f} / 5.0
    """, style='font-size: 20px;')

    # Add circle marker
    folium.CircleMarker(
        location=[row['center_lat'], row['center_lon']],
        radius=radius,
        color=assign_colour_on_severity(row['average severity']),
        fill=True,
        fill_color=assign_colour_on_severity(row['average severity']),
        fill_opacity=0.6,
        opacity=0.6,
        tooltip=tooltip_text,
    ).add_to(lobitosGrid)


lobitosGrid